# Example 1: Getting Started

A minimal end-to-end workflow that selects **4 representative months** from a year of hourly time-series data.

This example walks through the five pillars of the `energy-repset` framework:

| Pillar | Component | Choice in this example |
|--------|-----------|------------------------|
| **F** — Feature Space | How periods are compared | Statistical summaries (mean, std, min, max, quantiles, ramps) |
| **O** — Objective | What "representative" means | Wasserstein distance (marginal distribution fidelity) |
| **S** — Selection Space | What we pick from | All 4-of-12 monthly combinations (495 candidates) |
| **R** — Representation | How selected periods stand in for the year | Uniform weights (each month = 1/4 of the year) |
| **A** — Search Algorithm | How we find the best selection | Exhaustive generate-and-test with weighted-sum policy |

In [ ]:
import pandas as pd
import energy_repset as rep
import energy_repset.diagnostics as diag

## Load data

One year of hourly time series with four variables: electricity demand (`load`), onshore wind (`onwind`), offshore wind (`offwind`), and solar capacity factors (`solar`).

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
df_raw = pd.read_csv(url, index_col=0, parse_dates=True).rename_axis('variable', axis=1)
df_raw = df_raw.drop('prices', axis=1)
df_raw

## Define the problem context

The `TimeSlicer` divides the year into candidate periods — here, **12 calendar months**. The `ProblemContext` bundles the raw data and slicing logic into a single object that flows through the entire pipeline.

In [ ]:
slicer = rep.TimeSlicer(unit="month")
context = rep.ProblemContext(df_raw=df_raw, slicer=slicer)
print(f"Candidate slices: {context.get_unique_slices()}")

## Pillar F: Feature engineering

Before we can compare months, we need a numerical representation. `StandardStatsFeatureEngineer` computes a set of statistical summaries (mean, std, min, max, quantiles, ramp rates) per variable per month. This transforms each month into a fixed-length feature vector.

In [ ]:
feature_engineer = rep.StandardStatsFeatureEngineer()

## Pillar O: Objective

We use a single score component: **Wasserstein fidelity**. It measures how well the marginal distribution of the selected months matches the full year. Lower distance = better match.

With only one objective, the selection policy is straightforward — just pick the combination with the best score.

In [ ]:
objective_set = rep.ObjectiveSet({
    'wasserstein': (1.0, rep.WassersteinFidelity()),
})

## Pillars S + A: Selection space and search

`ExhaustiveCombiGen` enumerates all $\binom{12}{4} = 495$ ways to pick 4 months from 12. Each candidate is scored by the objective, and `WeightedSumPolicy` (trivial here with one component) picks the winner.

In [ ]:
k = 4
combi_gen = rep.ExhaustiveCombiGen(k=k)
policy = rep.WeightedSumPolicy()
search_algorithm = rep.ObjectiveDrivenCombinatorialSearchAlgorithm(
    objective_set, policy, combi_gen
)

## Pillar R: Representation model

With uniform weights, each selected month represents exactly 1/4 of the year. This is the simplest model — no cluster assignment, no optimization of weights. It places the full burden on the *selection itself* being intrinsically representative.

In [ ]:
representation_model = rep.UniformRepresentationModel()

## Run the workflow

In [ ]:
workflow = rep.Workflow(feature_engineer, search_algorithm, representation_model)
experiment = rep.RepSetExperiment(context, workflow)
result = experiment.run()

## Inspect results

In [ ]:
print(f"Selected months: {result.selection}")
print(f"Weights: {result.weights}")
print(f"Wasserstein score: {result.scores['wasserstein']:.4f}")

## Diagnostic: responsibility weights

The bar chart below shows the weight assigned to each selected month. With uniform representation, all bars are equal at 0.25. The dashed line indicates the "ideal" uniform reference.

In [ ]:
fig = diag.ResponsibilityBars().plot(result.weights, show_uniform_reference=True)
fig.update_layout(title='Responsibility Weights (Uniform)')
fig.show()